In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import csv
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [ ]:
def load_and_clean_html(directory_name):
    documents = []  # Üres lista a tisztított dokumentumok tárolására

    # Végigmegyünk a könyvtárban található összes fájlon
    for filename in os.listdir(directory_name):
        # Csak a .html kiterjesztésű fájlokat dolgozzuk fel
        if filename.endswith('.html'):
            # Biztonságos fájlnyitás UTF-8 kódolással
            with open(os.path.join(directory_name, filename), 'r', encoding='utf-8') as file:
                html = file.read()  # Beolvassuk a HTML fájl teljes tartalmát

                # Létrehozunk egy BeautifulSoup objektumot a HTML elemzéséhez
                soup = BeautifulSoup(html, 'html.parser')

                # Kinyerjük a tiszta szöveget (eltávolítva minden HTML tag-et)
                text = soup.get_text()

                # Hozzáadjuk a tisztított szöveget a dokumentumlistához
                documents.append(text)

    return documents  # Visszaadjuk a tisztított dokumentumok listáját

In [ ]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]{3,}')

In [ ]:
def get_frequent_words(documents, stop_count=100, corpus_count=10000):

    tokens = []  # Üres lista a tokenek (szavak) tárolására

    # Minden dokumentum feldolgozása a listában
    for doc in documents:
        # Tokenizálja a dokumentumot (szavakra bontja) és kisbetűssé alakít, majd hozzáadja a listához
        tokens.extend([word.lower() for word in tokenizer.tokenize(doc)])

    # Megszámolja minden szó előfordulási gyakoriságát
    word_counts = Counter(tokens)

    # Szavak szűrése:
    # 1. Kihagyja a legelső 'stop_count' darab leggyakoribb szót (általában kötőszavak)
    # 2. Kiválasztja a következő 'corpus_count' darab szót
    filtered = word_counts.most_common()[stop_count:stop_count + corpus_count]

    # Kinyeri csak a szavakat (a gyakoriság nélkül) a szűrt eredményekből
    words = [word for word, _ in filtered]

    return words

In [ ]:
documents = load_and_clean_html(".")

In [ ]:
words = get_frequent_words(documents)

In [ ]:
len(words)

10000

In [ ]:
word_to_id = {word: idx for idx, word in enumerate(words)} #Minden szóhoz rendelünk egy ID-t

In [ ]:
def generate_samples(documents, word_to_id, window_size=3):
    # Az ablak pozícióinak meghatározása (pl. [-3, -2, -1, 1, 2, 3] window_size=3 esetén)
    window_pos = [i for i in range(-window_size, window_size + 1) if i != 0]

    # Üres lista a minták tárolására
    samples = []

    # Szavak halmaza a gyorsabb keresés érdekében
    words_set = set(word_to_id.keys())

    # Végigmegyünk minden dokumentumon
    for document in documents:
        # Mondatokra bontjuk a dokumentumot
        for sentence in sent_tokenize(document):
            # Tokenizálás és kisbetűsítés, csak a szótárban lévő szavakat megtartjuk
            tokens = [word.lower() for word in tokenizer.tokenize(sentence) if word.lower() in words_set]

            # Végigmegyünk minden szón a mondaton belül
            for i, current in enumerate(tokens):
                # Megnézzük az összes lehetséges ablakpozíciót
                for pos in window_pos:
                    j = i + pos  # Számoljuk a szomszéd pozícióját

                    # Ellenőrizzük, hogy a pozíció érvényes-e (nem lóg ki a mondatból)
                    if 0 <= j < len(tokens):
                        # Hozzáadjuk a mintát (központi_szó, kontextus_szó) ID párokkal
                        samples.append((word_to_id[current], word_to_id[tokens[j]]))

    return samples

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
samples = generate_samples(documents, word_to_id)

In [ ]:
if torch.cuda.is_available():
    print('Using CUDA')
    device = torch.device('cuda')
else:
    print('Using CPU')
    device = torch.device('cpu')

Using CUDA


In [ ]:
def get_onehot_tensor(tensor):
    # Kinyerjük a tenzor első dimenziójának méretét (azaz hány szó van)
    size = [*tensor.shape][0]  # Például ha tensor.shape = (5,), akkor size = 5

    # Létrehozunk egy nullákkal teli tenzort a megfelelő méretben:
    # - Sorok száma: size (ennyi szó van)
    # - Oszlopok száma: len(word_to_id) (ennyi különböző szó lehet)
    inp = torch.zeros(size, len(word_to_id))

    # One-hot kódolás létrehozása a scatter_ függvénnyel:
    # - 1-es értékeket helyez el a megfelelő pozíciókba
    # - tensor.unsqueeze(1): Átalakítja a tenzort oszlopvektorrá (pl. [1,2,3] -> [[1],[2],[3]])
    # - 1.: Az érték amit be akarunk állítani (mindig 1-es)
    inp = inp.scatter_(1, tensor.unsqueeze(1), 1.)

    # Átmozgatjuk a tenzort a megfelelő eszközre (CPU vagy GPU)
    inp = inp.to(device)

    # Visszaadjuk a tenzort Variable formában és float típusként
    return Variable(inp).float()

In [ ]:
def one_hot_word(word, word_map):
    # Létrehozunk egy nullákkal teli vektort, ami annyi elemű, ahány szó van a szótárban
    one_hot_vector = np.zeros(len(word_map))

    # Beállítjuk az 1-es értéket a szónak megfelelő pozícióra
    # A word_map[word] megadja a szó indexét a szótárban
    one_hot_vector[word_map[word]] = 1

    # Visszaadjuk a kész one-hot vektort
    return one_hot_vector

In [ ]:
# A Word2vecDataset osztály egy egyedi PyTorch Dataset osztály,
# amely segít a Word2Vec modell tanításához szükséges adatok kezelésében.
class Word2vecDataset(Dataset):

    # Konstruktor: betölti a bemeneti adatokat és paramétereket
    def __init__(self, X, y, batch_size, word_map):
        # Bemeneti és cél (középső szó) adatok mentése
        self.X, self.y = X, y

        # Batch méret eltárolása
        self.batch_size = batch_size

        # Szóhoz tartozó indexek leképzése (word_map: pl. {"kutya": 0, "macska": 1, ...})
        self.word_map = word_map

    # Megadja, hogy hány batch van összesen az adathalmazban
    def __len__(self):
        # Összes X minta / batch méret, felfelé kerekítve (ha nem osztható maradék nélkül)
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    # Egy adott batch-hez tartozó X és y párok visszaadása
    def __getitem__(self, batch_id):
        # batch_id alapján kivágjuk az adott batch X elemeit (pl. szomszédos szavak indexei)
        batch_X = [i for i in self.X[batch_id * self.batch_size:(batch_id + 1) * self.batch_size]]

        # batch_id alapján kivágjuk az adott batch y elemeit (pl. középső szó indexei)
        batch_y = [i for i in self.y[batch_id * self.batch_size:(batch_id + 1) * self.batch_size]]

        # Mindkettőt one-hot tenzorrá alakítjuk és visszaadjuk
        return get_onehot_tensor(torch.tensor(batch_X)), get_onehot_tensor(torch.tensor(batch_y))

In [ ]:
# Hányszor tanulja végig a neurális hálózat az összes tanító adatot
epochs = 10
#Ez azt jelenti, hogy 10-szer végigmegy az összes batch-en (teljes tanulási ciklus)

# Hány példát dolgozunk fel egyszerre egy tanítási lépésben
batch_size = 1000
#Ez azt jelenti, hogy egyszerre 1000 szó-környezet párt használunk fel a tanításhoz

# Az embedding vektorok mérete (dimenziószáma)
embedding_dimension = 50
#A tanult szavak 50-dimenziós vektorként lesznek reprezentálva (ezek lesznek a "beágyazott" szóvektorok)

# A tanulási ráta (learning rate) a gradiens alapú optimalizáláshoz
learning_rate = 0.001
#A kis érték azt jelzi, hogy lassabban, de stabilabban tanul a modell (a súlyfrissítések mértéke)

#A kontextusablak mérete a Word2Vec modellben
window_size = 3
#Ez azt jelenti, hogy a középső szótól balra és jobbra max. 3 szót vesz figyelembe a környezetként (pl. [s1, s2, **közép**, s3, s4])

In [ ]:
X, y = zip(*samples)

# A Word2vecDataset példányosítása: létrehozunk egy tanító adatokat kezelő objektumot
dataset = Word2vecDataset(list(X), list(y), batch_size, word_to_id)
# A tanításhoz szükséges DataLoader létrehozása
train_dataloader = DataLoader(dataset)

# Első súlymátrix (W1) inicializálása: minden szónak lesz egy 50-dimenziós beágyazása
W1 = Variable(torch.randn(len(word_to_id), embedding_dimension, device=device).uniform_(0, 0.1).float(), requires_grad=True)
# Második súlymátrix (W2) inicializálása: visszavetítés az output rétegre
W2 = Variable(torch.randn(embedding_dimension, len(word_to_id), device=device).uniform_(0, 0.1).float(), requires_grad=True)

# Optimalizáló létrehozása: SGD (stochastic gradient descent)
optimizer = optim.SGD([W1, W2], lr=learning_rate)

# Megjelenítjük a két súlymátrixot (beágyazások és output súlyok)
W1, W2

(tensor([[0.0915, 0.0681, 0.0461,  ..., 0.0581, 0.0052, 0.0978],
         [0.0948, 0.0865, 0.0473,  ..., 0.0883, 0.0226, 0.0731],
         [0.0781, 0.0315, 0.0723,  ..., 0.0746, 0.0235, 0.0351],
         ...,
         [0.0527, 0.0923, 0.0482,  ..., 0.0114, 0.0333, 0.0988],
         [0.0302, 0.0469, 0.0106,  ..., 0.0761, 0.0134, 0.0843],
         [0.0078, 0.0329, 0.0640,  ..., 0.0506, 0.0890, 0.0602]],
        device='cuda:0', requires_grad=True),
 tensor([[0.0491, 0.0389, 0.0167,  ..., 0.0021, 0.0850, 0.0116],
         [0.0965, 0.0237, 0.0758,  ..., 0.0671, 0.0643, 0.0746],
         [0.0988, 0.0682, 0.0718,  ..., 0.0266, 0.0309, 0.0862],
         ...,
         [0.0703, 0.0514, 0.0882,  ..., 0.0986, 0.0671, 0.0828],
         [0.0170, 0.0170, 0.0654,  ..., 0.0872, 0.0508, 0.0037],
         [0.0732, 0.0471, 0.0241,  ..., 0.0166, 0.0576, 0.0620]],
        device='cuda:0', requires_grad=True))

In [ ]:
start = time.time()

# Végigmegyünk az epoch-okon (hányszor tanulja végig a modellt az adatokon)
for epoch in range(epochs):
  # Minden egyes batch-en végrehajtjuk a tanítást
    for inputs, outputs in train_dataloader:

        inputs = inputs[0].to(device)
        outputs = outputs[0].to(device)

        # Nullázzuk a gradiens értékeket (fontos minden batch előtt!)
        optimizer.zero_grad()

        # Az első réteg: input (one-hot) × W1 = beágyazott reprezentációk
        features = inputs.mm(W1)
        # A második réteg: beágyazások × W2 = kimeneti logitok
        y_out = features.mm(W2)

        # Keresztentrópia veszteségfüggvény inicializálása
        loss = torch.nn.CrossEntropyLoss()
        # A tényleges veszteség kiszámítása
        l = loss(y_out, outputs)
        # Visszaterjesztés (gradiens kiszámítása)
        l.backward()

        # A súlyok frissítése a gradiens irányában
        optimizer.step()

    print(f'Epoch {epoch+1}, loss = {l}')

print('Ellapsed hours: ', (time.time() - start)/3600)

Epoch 1, loss = 9.208711624145508
Epoch 2, loss = 9.208292007446289
Epoch 3, loss = 9.20787239074707
Epoch 4, loss = 9.207453727722168
Epoch 5, loss = 9.207035064697266
Epoch 6, loss = 9.20661449432373
Epoch 7, loss = 9.206195831298828
Epoch 8, loss = 9.20577621459961
Epoch 9, loss = 9.20535659790039
Epoch 10, loss = 9.204936027526855
Ellapsed hours:  0.6850854672325982


In [ ]:
def get_sigmoid_predictions(word):
    if word not in word_to_id:
        return ['Could not predict context']

    # One-hot enkódolás
    x = one_hot_word(word, word_to_id)
    x = torch.from_numpy(np.array(x)).unsqueeze(0).to(device).float()

    # Feed forward
    features = x.mm(W1)
    y_out = features.mm(W2)

    # Sigmoid
    return torch.sigmoid(y_out)

In [ ]:
def get_binary_context(sigmoid_output, threshold=0.5):
    # Átalakítás numpy arrayé
    sigmoid_values = sigmoid_output.detach().cpu().numpy()[0]

    # Fordított szótár létrehozása
    inverted_vocab = {v: k for k, v in word_to_id.items()}

    # Kontextus szavak kiválasztása a küszöbérték alapján
    context_words = []
    for idx, val in enumerate(sigmoid_values):
        if val > threshold:
            context_words.append(inverted_vocab[idx])

    return context_words, sigmoid_values

In [ ]:
sigmoid_output = get_sigmoid_predictions('male')
context_words, sigmoid_values = get_binary_context(sigmoid_output)
print("Kontextus szavak:", context_words)
print("Sigmoid értékek:", sigmoid_values)

Kontextus szavak: ['way', 'over', 'without', 'life', 'king', 'lady', 'take', 'don', 'even', 'why', 'away', 'thought', 'hand', 'nothing', 'thee', 'young', 'house', 'give', 'eyes', 'long', 'those', 'tell', 'came', 'dear', 'ever', 'look', 'every', 'night', 'last', 'still', 'room', 'many', 'work', 'myself', 'heart', 'miss', 'head', 'went', 'father', 'men', 'same', 'once', 'back', 'poor', 'while', 'place', 'another', 'face', 'its', 'off', 'through', 'friend', 'found', 'gutenberg', 'put', 'whom', 'enter', 'hath', 'seemed', 'indeed', 'mind', 'better', 'having', 'just', 'project', 'world', 'people', 'always', 'yes', 'saw', 'part', 'mother', 'looked', 'things', 'speak', 'under', 'both', 'something', 'against', 'done', 'took', 'quite', 'told', 'nor', 'name', 'right', 'going', 'heard', 'find', 'get', 'door', 'woman', 'left', 'god', 'thing', 'hear', 'three', 'soon', 'herself', 'whose', 'home', 'got', 'chapter', 'set', 'asked', 'new', 'gentleman', 'scene', 'till', 'seen', 'death', 'knew', 'cannot',

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
feature_names = vectorizer.get_feature_names_out().tolist()

In [ ]:
def cosine_distance(vector1, vector2):
  # Kiszámítjuk a két vektor skaláris szorzatát (dot product)
    # Ez adja meg, mennyire "mutatnak ugyanabba az irányba"
    # Kiszámítjuk az első vektor hosszát (normáját, azaz abszolút értékét)
    # Kiszámítjuk a második vektor hosszát (normáját)
    # A koszinusz-hasonlóság képlete: (v1 · v2) / (||v1|| * ||v2||)
    # Ez 1, ha a vektorok teljesen azonos irányúak, 0 ha merőlegesek
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [ ]:
def tfidf_cosine_distance(word1, word2):
    # Megkapjuk az első szóhoz tartozó TF-IDF vektort
    tfidf_word1 = tfidf_matrix[:, feature_names.index(word1)].toarray().flatten()
    # Megkapjuk a második szóhoz tartozó TF-IDF vektort
    tfidf_word2 = tfidf_matrix[:, feature_names.index(word2)].toarray().flatten()

    return cosine_distance(tfidf_word1, tfidf_word2)

In [ ]:
def word2vec_cosine_distance(word1, word2):
  # Lekérjük az első szóhoz tartozó kimeneti szigmoid értékeket a Word2Vec hálóból
    # A kimenet: tensor, amelyet előbb áthelyezünk CPU-ra, leválasztunk a gráfról (.detach()),
    # majd numpy tömbbé alakítunk, és kivesszük az első sort ([0])
    s1 = get_sigmoid_predictions(word1).cpu().detach().numpy()[0]
    # Ugyanezt megcsináljuk a második szóval is
    s2 = get_sigmoid_predictions(word2).cpu().detach().numpy()[0]


# Visszatérünk a két szó közötti koszinusz-hasonlósággal (Word2Vec kimenet alapján)
    return cosine_distance(s1, s2)

In [ ]:
# Létrehozunk egy üres szótárat, amelybe a hasonlósági eredményeket mentjük
similarities = {}

with open('combined.csv', 'r') as f:
    rows = csv.reader(f)
    next(rows)
    # Végigmegyünk minden soron, amely egy szó-párból és egy hasonlósági értékből áll
    for word1, word2, sim in rows:
      # Csak akkor számolunk, ha mindkét szó szerepel a word_to_id szótárban
        if word1 in words and word2 in words:
            similarities[(word1, word2)] = {
                'wordsim353': float(sim), # Az emberi annotáció alapján mért hasonlóság (Wordsim-353 adatbázisból)
                'word2vec': word2vec_cosine_distance(word1, word2), # Word2Vec hálóból számított hasonlóság
                'bow': tfidf_cosine_distance(word1, word2) # Bag-of-Words (TF-IDF alapú) hasonlóság
            }

# Megkeressük a legkisebb és legnagyobb értéket a wordsim353 (emberi értékelések) alapján
min = np.min([i['wordsim353'] for i in similarities.values()])
max = np.max([i['wordsim353'] for i in similarities.values()])

# Normalizáljuk az emberi hasonlósági értékeket 0 és 1 közé, hogy összehasonlíthatók legyenek
for word_pair in similarities.values():
    word_pair['wordsim353'] = (word_pair['wordsim353'] - min) / (max - min)


# Visszatérünk a similarities szótárral, amely minden szó-párhoz tartalmaz háromféle hasonlóságot:
# 1. normált emberi értékelést (wordsim353),
# 2. Word2Vec-alapú hasonlóságot (word2vec),
# 3. TF-IDF (bag-of-words) alapú hasonlóságot (bow).
similarities

{('tiger', 'cat'): {'wordsim353': np.float64(0.7198731501057082),
  'word2vec': np.float32(0.99999315),
  'bow': np.float64(0.9255579335450445)},
 ('tiger', 'tiger'): {'wordsim353': np.float64(1.0),
  'word2vec': np.float32(1.0),
  'bow': np.float64(0.0)},
 ('book', 'paper'): {'wordsim353': np.float64(0.7315010570824524),
  'word2vec': np.float32(0.9999928),
  'bow': np.float64(0.5335554880017467)},
 ('plane', 'car'): {'wordsim353': np.float64(0.5528541226215644),
  'word2vec': np.float32(0.99999374),
  'bow': np.float64(0.658044515380798)},
 ('train', 'car'): {'wordsim353': np.float64(0.609936575052854),
  'word2vec': np.float32(0.9999925),
  'bow': np.float64(0.3669257107478645)},
 ('telephone', 'communication'): {'wordsim353': np.float64(0.7357293868921775),
  'word2vec': np.float32(0.99999356),
  'bow': np.float64(0.6463955676189458)},
 ('drug', 'abuse'): {'wordsim353': np.float64(0.6670190274841437),
  'word2vec': np.float32(0.99999225),
  'bow': np.float64(0.9805302581287035)},
 

In [ ]:
mean_squared_error(
    # Első lista: a normalizált emberi hasonlósági értékek (Wordsim353 alapján)
    [i['wordsim353'] for i in similarities.values()],
    # Második lista: a Word2Vec modell által becsült hasonlóságok
    [i['word2vec'] for i in similarities.values()],
)

0.2732612243431965

In [ ]:
mean_squared_error(
    # Az ember által adott hasonlósági értékek (Wordsim353), normalizálva 0 és 1 közé
    [i['wordsim353'] for i in similarities.values()],
    # A TF-IDF (Bag-of-Words) modell által számított hasonlósági értékek (valójában távolságok, de 0-hoz közel jobb)
    [i['bow'] for i in similarities.values()],
)

0.12106403069867468

In [ ]:
mean_squared_error(
    # A TF-IDF (Bag-of-Words) modell által számított hasonlósági értékek (valójában távolságok, tehát kisebb = hasonlóbb)
    [i['bow'] for i in similarities.values()],
    # A Word2Vec modell által számított hasonlósági értékek (szintén távolságként értelmezve)
    [i['word2vec'] for i in similarities.values()],
)

0.2369449154068647